In [1]:
import datetime
import pandas as pd
import geopandas as gpd

import FireMain, FireTime, FireObj, FireConsts, postprocess, preprocess
from utils import timed

In [2]:
tst = [2023, 8, 28, 'AM']
ted = [2023, 9, 6, 'AM']
region = ('WesternUS',[-125.698046875,31.676476158707615,
                       -101.00078125,49.51429477264348])
list_of_ts = list(FireTime.t_generator(tst, ted))

## Once per landcover file

If your landcover file is not in lat,lon then transform it and resave it. You will use this file a bunch later on.

In [ ]:
preprocess.preprocess_landcover()

## Once per region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [3]:
preprocess.preprocess_region(region)

2024-03-20 20:16:40,004 - FireLog - INFO - func:preprocess_region took: 40.61 sec


## Once per input file

Next process each NRT file into half day files. First we'll get all the times that are of interest. This could also be done by inspecting all looking at all the files that exist and seeing which have not been preprocessed yet.

In [4]:
%%time
for sat in ["SNPP", "NOAA20"]:
    for t in list_of_ts[::2]:
        preprocess.preprocess_NRT_file(t, sat)

2024-03-20 20:16:40,010 - FireLog - INFO - preprocessing NRT file for 2023-8-28, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:312: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
2024-03-20 20:16:41,605 - FireLog - INFO - func:preprocess_NRT_file took: 1.59 sec
2024-03-20 20:16:41,606 - FireLog - INFO - preprocessing NRT file for 2023-8-29, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:312: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
2024-03-20 20:16:43,064 - FireLog - INFO - func:preprocess_NRT_file took: 1.46 sec
2024-03-20 20:16:43,064 - FireLog - INFO - preprocessing NRT file for 2023-8-30, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:312: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is depr

CPU times: user 14.5 s, sys: 420 ms, total: 14.9 s
Wall time: 28.8 s


## Once per region and t

Do initial filtering and clustering using the preprocessed region and the half day files.

Note: for the purpose of timing I am running the full month, but each of these steps could run in a separate proccess.

In [5]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, sensor="VIIRS", region=region)

2024-03-20 20:17:08,771 - FireLog - INFO - filtering and clustering 2023-8-28 AM, VIIRS, WesternUS
2024-03-20 20:17:08,829 - FireLog - INFO - func:read_preprocessed took: 57.61 ms
2024-03-20 20:17:08,874 - FireLog - INFO - func:read_preprocessed took: 44.29 ms
2024-03-20 20:17:09,085 - FireLog - INFO - func:read_region took: 208.89 ms
2024-03-20 20:18:00,720 - FireLog - INFO - func:do_clustering took: 394.16 ms
2024-03-20 20:18:00,784 - FireLog - INFO - func:preprocess_region_t took: 52.01 sec
2024-03-20 20:18:00,784 - FireLog - INFO - filtering and clustering 2023-8-28 PM, VIIRS, WesternUS
2024-03-20 20:18:00,850 - FireLog - INFO - func:read_preprocessed took: 64.68 ms
2024-03-20 20:18:00,918 - FireLog - INFO - func:read_preprocessed took: 67.74 ms
2024-03-20 20:18:01,093 - FireLog - INFO - func:read_region took: 171.81 ms
2024-03-20 20:18:22,657 - FireLog - INFO - func:do_clustering took: 18.69 ms
2024-03-20 20:18:22,699 - FireLog - INFO - func:preprocess_region_t took: 21.91 sec
202

CPU times: user 4min 18s, sys: 323 ms, total: 4min 18s
Wall time: 4min 32s


## Run the fire expansion and merging algorithm

`Fire_Forward` is responsible for reading in the preprocessed data created in the `Ingest` notebook

In [6]:
allfires, allpixels = FireMain.Fire_Forward(tst=tst, ted=ted, restart=False, region=region)

2024-03-20 20:21:41,013 - FireLog - INFO - func:read_preprocessed took: 8.24 ms
2024-03-20 20:21:41,020 - FireLog - INFO - func:read_preprocessed took: 6.08 ms
2024-03-20 20:21:41,026 - FireLog - INFO - func:read_preprocessed took: 6.31 ms
2024-03-20 20:21:41,032 - FireLog - INFO - func:read_preprocessed took: 5.49 ms
2024-03-20 20:21:41,039 - FireLog - INFO - func:read_preprocessed took: 5.86 ms
2024-03-20 20:21:41,045 - FireLog - INFO - func:read_preprocessed took: 6.42 ms
2024-03-20 20:21:41,053 - FireLog - INFO - func:read_preprocessed took: 7.46 ms
2024-03-20 20:21:41,058 - FireLog - INFO - func:read_preprocessed took: 3.95 ms
2024-03-20 20:21:41,063 - FireLog - INFO - func:read_preprocessed took: 4.21 ms
2024-03-20 20:21:41,067 - FireLog - INFO - func:read_preprocessed took: 4.15 ms
2024-03-20 20:21:41,073 - FireLog - INFO - func:read_preprocessed took: 5.52 ms
2024-03-20 20:21:41,078 - FireLog - INFO - func:read_preprocessed took: 3.87 ms
2024-03-20 20:21:41,082 - FireLog - INFO

We can save these files off in case something goes wrong later

In [7]:
postprocess.save_allpixels(allpixels, tst, ted, region)
postprocess.save_allfires_gdf(allfires.gdf, tst, ted, region)

2024-03-20 20:22:36,826 - FireLog - INFO - func:save_allpixels took: 206.06 ms
2024-03-20 20:22:37,036 - FireLog - INFO - func:save_allfires_gdf took: 208.92 ms


## Write Snapshot files

Start by reading in the outputs from the run:

In [3]:
allpixels = postprocess.read_allpixels(tst, ted, region)
allfires_gdf = postprocess.read_allfires_gdf(tst, ted, region)

2024-03-20 20:23:11,766 - FireLog - INFO - func:read_allpixels took: 41.47 ms
2024-03-20 20:23:11,879 - FireLog - INFO - func:read_allfires_gdf took: 112.90 ms


Then iterate through all of the ts and save the snapshot layers.

In [4]:
%%time
postprocess.save_snapshots(allfires_gdf, region, tst, ted)

2024-03-20 20:23:14,054 - FireLog - INFO - func:save_snapshot_layers took: 477.00 ms
2024-03-20 20:23:14,552 - FireLog - INFO - func:save_snapshot_layers took: 493.32 ms
2024-03-20 20:23:15,116 - FireLog - INFO - func:save_snapshot_layers took: 559.62 ms
2024-03-20 20:23:15,658 - FireLog - INFO - func:save_snapshot_layers took: 537.54 ms
2024-03-20 20:23:16,252 - FireLog - INFO - func:save_snapshot_layers took: 590.23 ms
2024-03-20 20:23:16,858 - FireLog - INFO - func:save_snapshot_layers took: 602.23 ms
2024-03-20 20:23:17,465 - FireLog - INFO - func:save_snapshot_layers took: 602.80 ms
2024-03-20 20:23:18,119 - FireLog - INFO - func:save_snapshot_layers took: 649.83 ms
2024-03-20 20:23:18,734 - FireLog - INFO - func:save_snapshot_layers took: 610.33 ms
2024-03-20 20:23:19,389 - FireLog - INFO - func:save_snapshot_layers took: 651.16 ms
2024-03-20 20:23:20,054 - FireLog - INFO - func:save_snapshot_layers took: 661.04 ms
2024-03-20 20:23:20,760 - FireLog - INFO - func:save_snapshot_lay

CPU times: user 7.7 s, sys: 191 ms, total: 7.89 s
Wall time: 12.8 s


## Write Largefire files

First we will find all the large fires:

In [5]:
large_fires = postprocess.find_largefires(allfires_gdf)

2024-03-20 20:23:26,346 - FireLog - INFO - func:find_largefires took: 4.80 ms


First we'll use the `allpixels` object to create the `nplist` layer

In [6]:
%%time
postprocess.save_large_fires_nplist(allpixels, region, large_fires, tst)

2024-03-20 20:23:33,084 - FireLog - INFO - func:save_fire_nplist took: 45.78 ms
2024-03-20 20:23:33,127 - FireLog - INFO - func:save_fire_nplist took: 41.89 ms
2024-03-20 20:23:33,187 - FireLog - INFO - func:save_fire_nplist took: 58.72 ms
2024-03-20 20:23:33,241 - FireLog - INFO - func:save_fire_nplist took: 52.89 ms
2024-03-20 20:23:33,289 - FireLog - INFO - func:save_fire_nplist took: 46.89 ms
2024-03-20 20:23:33,338 - FireLog - INFO - func:save_fire_nplist took: 47.36 ms
2024-03-20 20:23:33,390 - FireLog - INFO - func:save_fire_nplist took: 51.62 ms
2024-03-20 20:23:33,433 - FireLog - INFO - func:save_fire_nplist took: 41.51 ms
2024-03-20 20:23:33,483 - FireLog - INFO - func:save_fire_nplist took: 49.14 ms
2024-03-20 20:23:33,523 - FireLog - INFO - func:save_fire_nplist took: 38.98 ms
2024-03-20 20:23:33,565 - FireLog - INFO - func:save_fire_nplist took: 40.72 ms
2024-03-20 20:23:33,617 - FireLog - INFO - func:save_fire_nplist took: 51.16 ms
2024-03-20 20:23:33,666 - FireLog - INFO

CPU times: user 919 ms, sys: 38.9 ms, total: 958 ms
Wall time: 2.01 s


The rest of the layers will be created directly from the `allfires_gdf` and have some merge handling

In [7]:
%%time
postprocess.save_large_fires_layers(allfires_gdf, region, large_fires, tst)

2024-03-20 20:23:39,277 - FireLog - INFO - func:save_fire_layers took: 135.38 ms


44 rows that potentially need a merge


2024-03-20 20:23:39,431 - FireLog - INFO - func:save_fire_layers took: 153.16 ms
2024-03-20 20:23:39,572 - FireLog - INFO - func:save_fire_layers took: 139.53 ms
2024-03-20 20:23:39,598 - FireLog - INFO - func:merge_rows took: 24.20 ms
2024-03-20 20:23:39,724 - FireLog - INFO - func:save_fire_layers took: 126.22 ms
2024-03-20 20:23:39,740 - FireLog - INFO - func:merge_rows took: 14.42 ms
2024-03-20 20:23:39,862 - FireLog - INFO - func:save_fire_layers took: 120.99 ms
2024-03-20 20:23:39,989 - FireLog - INFO - func:save_fire_layers took: 126.25 ms
2024-03-20 20:23:40,106 - FireLog - INFO - func:save_fire_layers took: 114.94 ms
2024-03-20 20:23:40,237 - FireLog - INFO - func:save_fire_layers took: 130.32 ms
2024-03-20 20:23:40,361 - FireLog - INFO - func:save_fire_layers took: 122.80 ms
2024-03-20 20:23:40,545 - FireLog - INFO - func:save_fire_layers took: 182.24 ms
2024-03-20 20:23:40,693 - FireLog - INFO - func:save_fire_layers took: 147.22 ms
2024-03-20 20:23:40,832 - FireLog - INFO -

CPU times: user 1.46 s, sys: 66.5 ms, total: 1.53 s
Wall time: 4.76 s
